In [1]:
import cv2
import torch
import numpy as np


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from utils.tensorboard import TensorBoard
from Renderer.model import FCN
from Renderer.stroke_gen import *

In [3]:
writer = TensorBoard("../train_log/rec ")
import torch.optim as optim

criterion = nn.MSELoss()
net = FCN()
optimizer = optim.Adam(net.parameters(), lr=3e-6)
batch_size = 64

use_cuda = torch.cuda.is_available()
step = 0

In [ ]:
def save_model():
    if use_cuda:
        net.cpu()
    torch.save(net.state_dict(), "./model/renderer_rec.pkl")
    if use_cuda:
        net.cuda()


def load_weights():
    pretrained_dict = torch.load("./model/renderer_rec.pkl")
    model_dict = net.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    net.load_state_dict(model_dict)


# load_weights()
# 500000
while step < 50000:
    net.train()
    train_batch = []
    ground_truth = []
    for i in range(batch_size):
        f = np.random.uniform(0, 1, 10)
        train_batch.append(f)
        ground_truth.append(draw_rec(f))

    train_batch = torch.tensor(train_batch).float()
    ground_truth = torch.tensor(ground_truth).float()
    if use_cuda:
        net = net.cuda()
        train_batch = train_batch.cuda()
        ground_truth = ground_truth.cuda()
    gen = net(train_batch)
    optimizer.zero_grad()
    loss = criterion(gen, ground_truth)
    loss.backward()
    optimizer.step()
    print(step, loss.item())
#     if step < 200000:
#         lr = 1e-4
#     elif step < 400000:
#         lr = 1e-5
#     else:
#         lr = 1e-6
    if step < 20000:
        lr = 1e-4
    elif step < 40000:
        lr = 1e-5
    else:
        lr = 1e-6
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr
    writer.add_scalar("train_rec/loss", loss.item(), step)
    if step % 10 == 0:
        net.eval()
        gen = net(train_batch)
        loss = criterion(gen, ground_truth)
        writer.add_scalar("val_rec/loss", loss.item(), step)
        for i in range(32):
            G = gen[i].cpu().data.numpy()
            GT = ground_truth[i].cpu().data.numpy()
            writer.add_image("train_rec/gen{}.png".format(i), G, step)
            writer.add_image("train_rec/ground_truth{}.png".format(i), GT, step)
    if step % 100 == 0:
        save_model()
    step += 1

0 0.20161043107509613
1 0.20138269662857056
2 0.19897155463695526
3 0.19400516152381897
4 0.19811345636844635
5 0.19875521957874298
6 0.1988469958305359
7 0.1926981657743454
8 0.1980656087398529
9 0.20205280184745789
10 0.1945468634366989
11 0.19976943731307983
12 0.19367286562919617
13 0.19687213003635406
14 0.19353076815605164
15 0.20006805658340454
16 0.20371252298355103
17 0.19929201900959015
18 0.19743633270263672
19 0.20013098418712616
20 0.1977366954088211
21 0.20241013169288635
22 0.20264606177806854
23 0.2006499171257019
24 0.19898709654808044
25 0.19844168424606323
26 0.20140519738197327
27 0.20040950179100037
28 0.1956462562084198
29 0.19616225361824036
30 0.19800561666488647
31 0.19906304776668549
32 0.20214040577411652
33 0.1972856968641281
34 0.19591087102890015
35 0.19115114212036133
36 0.19888192415237427
37 0.20361274480819702
38 0.19528818130493164
39 0.19025784730911255
40 0.19647935032844543
41 0.19769629836082458
42 0.19702237844467163
43 0.20194721221923828
44 0.1

347 0.05134698003530502
348 0.055498309433460236
349 0.04123731702566147
350 0.05689620599150658
351 0.057223282754421234
352 0.05736083909869194
353 0.05135517939925194
354 0.04402798414230347
355 0.0494915246963501
356 0.06556062400341034
357 0.05256078019738197
358 0.046658601611852646
359 0.050266630947589874
360 0.04043443873524666
361 0.06360406428575516
362 0.057691674679517746
363 0.05215620622038841
364 0.051093198359012604
365 0.053537048399448395
366 0.05730900913476944
367 0.053668275475502014
368 0.057560913264751434
369 0.05177164450287819
370 0.04317300394177437
371 0.06283856928348541
372 0.04346147179603577
373 0.04630487412214279
374 0.047887206077575684
375 0.05017290636897087
376 0.04782496765255928
377 0.04123900085687637
378 0.05295421555638313
379 0.057255107909440994
380 0.04881498962640762
381 0.036873959004879
382 0.06118590757250786
383 0.061217937618494034
384 0.04988228902220726
385 0.05665580555796623
386 0.05091888830065727
387 0.06648864597082138
388 0.0

686 0.034893207252025604
687 0.02779904566705227
688 0.0344155989587307
689 0.026170657947659492
690 0.03278126195073128
691 0.030047573149204254
692 0.025676719844341278
693 0.035393014550209045
694 0.031489983201026917
695 0.023978080600500107
696 0.025332167744636536
697 0.025189565494656563
698 0.027296924963593483
699 0.027672503143548965
700 0.03063495084643364
701 0.028901373967528343
702 0.02555633708834648
703 0.028106926009058952
704 0.025494158267974854
705 0.03000108152627945
706 0.02847401238977909
707 0.02309422940015793
708 0.027522237971425056
709 0.02796272188425064
710 0.027848582714796066
711 0.026771649718284607
712 0.02981012873351574
713 0.023920675739645958
714 0.030536925420165062
715 0.025291061028838158
716 0.03255018964409828
717 0.0316418819129467
718 0.029143627732992172
719 0.025788921862840652
720 0.025594504550099373
721 0.02386663481593132
722 0.030422914773225784
723 0.03436436131596565
724 0.026927489787340164
725 0.0389486663043499
726 0.030215186998

1019 0.030668681487441063
1020 0.026096850633621216
1021 0.02252226322889328
1022 0.02651326172053814
1023 0.027093205600976944
1024 0.027624882757663727
1025 0.027921181172132492
1026 0.024564852938055992
1027 0.03183864429593086
1028 0.0315096341073513
1029 0.032990630716085434
1030 0.03195007145404816
1031 0.02138633280992508
1032 0.024558018893003464
1033 0.028244677931070328
1034 0.03341913968324661
1035 0.025470834225416183
1036 0.023562561720609665
1037 0.027262497693300247
1038 0.025085972622036934
1039 0.022079451009631157
1040 0.02538493275642395
1041 0.025126807391643524
1042 0.031081538647413254
1043 0.03047320246696472
1044 0.026165848597884178
1045 0.025930847972631454
1046 0.024321841076016426
1047 0.021709870547056198
1048 0.028440823778510094
1049 0.023144759237766266
1050 0.023656122386455536
1051 0.031913504004478455
1052 0.022216880694031715
1053 0.019459005445241928
1054 0.025018537417054176
1055 0.023901769891381264
1056 0.020164143294095993
1057 0.029660437256097

1339 0.026722567155957222
1340 0.02939208596944809
1341 0.025253552943468094
1342 0.021350298076868057
1343 0.025056147947907448
1344 0.025606151670217514
1345 0.03104272671043873
1346 0.02346733585000038
1347 0.02977442741394043
1348 0.02167835831642151
1349 0.02465679496526718
1350 0.024548927322030067
1351 0.02339651808142662
1352 0.025303401052951813
1353 0.021914605051279068
1354 0.020399590954184532
1355 0.02586197480559349
1356 0.021822521463036537
1357 0.02939232438802719
1358 0.02290491759777069
1359 0.02074405923485756
1360 0.02315530739724636
1361 0.029149923473596573
1362 0.023928025737404823
1363 0.02337804064154625
1364 0.02362053282558918
1365 0.021893711760640144
1366 0.027026250958442688
1367 0.024911094456911087
1368 0.0224042646586895
1369 0.028115594759583473
1370 0.027338609099388123
1371 0.025416675955057144
1372 0.026320159435272217
1373 0.025188034400343895
1374 0.020841393619775772
1375 0.024795038625597954
1376 0.020529022440314293
1377 0.02730143815279007
137

...

48533 0.0020762167405337095
48534 0.0017794992309063673
48535 0.0016940326895564795
48536 0.0019249356118962169
48537 0.0015070321969687939
48538 0.0014603649033233523
48539 0.001710437354631722
48540 0.0014781149802729487
48541 0.0018473602831363678
48542 0.001625492936000228
48543 0.0018608302343636751
48544 0.0017639521975070238
48545 0.0015494449762627482
48546 0.0015479481080546975
48547 0.0016379011794924736
48548 0.0019673898350447416
48549 0.0016744118183851242
48550 0.0017715701833367348
48551 0.0017214594408869743
48552 0.0016795448027551174
48553 0.001862811390310526
48554 0.0017865521367639303
48555 0.001920565264299512
48556 0.0023926186840981245
48557 0.0016929717967286706
48558 0.0015128104714676738
48559 0.0015807574382051826
48560 0.001822293153963983
48561 0.002089825691655278
48562 0.0018002920551225543
48563 0.0022407337091863155
48564 0.0018529705703258514
48565 0.0017474257620051503
48566 0.0019825741183012724
48567 0.002280802233144641
48568 0.0016679707914590836

48829 0.0021402218844741583
48830 0.001750683644786477
48831 0.0016401279717683792
48832 0.001813756301999092
48833 0.0019268899923190475
48834 0.0018326480640098453
48835 0.0019692336209118366
48836 0.0018256697803735733
48837 0.0016878419555723667
48838 0.0016942528309300542
48839 0.0019500473281368613
48840 0.0018518469296395779
48841 0.00215859804302454
48842 0.0018773623742163181
48843 0.0020623349118977785
48844 0.0019585508853197098
48845 0.0016641131369397044
48846 0.0018581647891551256
48847 0.0017395906616002321
48848 0.0017654552357271314
48849 0.0017012815224006772
48850 0.0017491617472842336
48851 0.0016743630403652787
48852 0.0021687413100153208
48853 0.0016855158610269427
48854 0.001790722832083702
48855 0.0015180662740021944
48856 0.0020372497383505106
48857 0.0016379212029278278
48858 0.002011071890592575
48859 0.0019036771263927221
48860 0.0016902759671211243
48861 0.0019084428204223514
48862 0.0017890651943162084
48863 0.0021116160787642
48864 0.0017369247507303953
4

49125 0.0019660687539726496
49126 0.001575622707605362
49127 0.0017941738478839397
49128 0.001970059471204877
49129 0.0020342532079666853
49130 0.0018784553976729512
49131 0.0016682922141626477
49132 0.002410896122455597
49133 0.0017334206495434046
49134 0.0015118158189579844
49135 0.0019845745991915464
49136 0.00173379376064986
49137 0.001872250810265541
49138 0.001911580329760909
49139 0.0019028177484869957
49140 0.002104841871187091
49141 0.0016011686529964209
49142 0.0018901623552665114
49143 0.0019666708540171385
49144 0.0018108133226633072
49145 0.0017893349286168814
49146 0.001960344146937132
49147 0.0018107055220752954
49148 0.0018873793305829167
49149 0.001903496333397925
49150 0.0018685582326725125
49151 0.0018426894675940275
49152 0.001621794537641108
49153 0.0019562686793506145
49154 0.0017353720031678677
49155 0.0017911610193550587
49156 0.0018538818694651127
49157 0.0019941770005971193
49158 0.0018655966268852353
49159 0.0018151976400986314
49160 0.0017351913265883923
491

49421 0.001920073525980115
49422 0.001943287905305624
49423 0.0018338579684495926
49424 0.0021209586411714554
49425 0.001864774851128459
49426 0.0017099917167797685
49427 0.0018424896989017725
49428 0.0017716668080538511
49429 0.001811120891943574
49430 0.0017670849338173866
49431 0.0016929247649386525
49432 0.0019818933214992285
49433 0.0018859029514715075
49434 0.0017972900532186031
49435 0.0016325018368661404
49436 0.0017106038285419345
49437 0.0018376620719209313
49438 0.001603337237611413
49439 0.0018433509394526482
49440 0.0017664664192125201
49441 0.0017046494176611304
49442 0.0018341690301895142
49443 0.0019395409617573023
49444 0.0017305059591308236
49445 0.002015336649492383
49446 0.002010196913033724
49447 0.0019265342270955443
49448 0.0016473543364554644
49449 0.0019337674602866173
49450 0.0020443638786673546
49451 0.0017811612924560905
49452 0.0018313006730750203
49453 0.0019000049214810133
49454 0.002003235975280404
49455 0.0019129958236590028
49456 0.002107600448653102
4

49717 0.0019894966389983892
49718 0.0015579707687720656
49719 0.0020796915050596
49720 0.0016698995605111122
49721 0.0014111902564764023
49722 0.0017853452591225505
49723 0.0017908569425344467
49724 0.0016339978901669383
49725 0.002088276669383049
49726 0.0016153292963281274
49727 0.0017829752760007977
49728 0.0015592121053487062
49729 0.0014162971638143063
49730 0.0015909203793853521
49731 0.0016877332236617804
49732 0.0015514036640524864
49733 0.0017366171814501286
49734 0.001987406052649021
49735 0.002185427350923419
49736 0.0016747108893468976
49737 0.0020099615212529898
49738 0.0019482698990032077
49739 0.0018994766287505627
49740 0.002002839930355549
49741 0.0020959125831723213
49742 0.0019667295273393393
49743 0.0020507744047790766
49744 0.001620202325284481
49745 0.0018602353520691395
49746 0.001714975223876536
49747 0.0017715345602482557
49748 0.0018692933954298496
49749 0.0016507335240021348
49750 0.0016518847551196814
49751 0.001900962321087718
49752 0.0017438780050724745
49